In [8]:
from rep_concat_scheme import *
from tqdm import tqdm
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

In [9]:
def xzrepconc(mx,mz,px_0,py_0,pz_0):
    concatenated_rate = 0
    for rx in range(0, mx):
        for rz in range(0, mz):
            #new channel after z repetition: 
            pI_1,px_1,py_1,pz_1 = new_channel(rz,mz,px_0,py_0,pz_0)
            probability_z_syndrome = probability_of_a_syndrome(rz, mz, px_0,py_0,pz_0)
            
            #for x-repetition code switch px and pz
            px_2 = pz_1
            pz_2 = px_1
            
            p_syndrome_x_z = probability_of_a_syndrome(rx, mx, px_2,py_1,pz_2)*probability_z_syndrome
            coefficient = binom(mx - 1, rx)*binom(mz - 1, rz)
            
            concatenated_rate += coefficient * p_syndrome_x_z * max(0, rate(new_channel(rx, mx,px_2,py_1,pz_2)))
    return concatenated_rate / (mx*mz)

In [10]:
def xz_get_best_rate(pX, pY, pZ, max_mx, max_mz):
    pvec = [1 - pX - pY - pZ, pX, pY, pZ]
    best_rate = rate(pvec)
    for mx in range(2, max_mx):
        for mz in range(2, max_mz):
            r = xzrepconc(mx,mz,pX, pY, pZ)
            if r >= best_rate:
                best_rate = r
    
    return max(best_rate,get_best_rate(pX, pY, pZ, max_mz))


In [11]:
fidelity_range = np.arange(0.5,1,0.01)
RC = []
RCxz = []
max_rep = 5
for f in tqdm(fidelity_range):
    p_pauli = (1-f)/3
    RC.append(get_best_rate(p_pauli,p_pauli,p_pauli,max_rep*3))
    RCxz.append(xz_get_best_rate(p_pauli, p_pauli, p_pauli, max_rep, 2*max_rep))

100%|██████████| 50/50 [00:07<00:00,  7.07it/s]


In [12]:
plt.interactive(False)

plt.plot(fidelity_range,RCxz,label='xz')
plt.plot(fidelity_range,RC,label='z')
plt.legend()
plt.show(block=True)